### Imports

In [4]:
import numpy as np
import pandas as pd
import fastwer
import os
from tqdm import tqdm

### Functions

In [5]:
def calculate_iou(box1, box2):
    """
    Calculate IOU between two bounding boxes.
    box1: first bounding box (x1, y1, x2, y2)
    box2: second bounding box (x1, y1, x2, y2)
    """
    # determine the coordinates of the intersection rectangle
    x_left = max(box1[0], box2[0])
    y_top = max(box1[1], box2[1])
    x_right = min(box1[2], box2[2])
    y_bottom = min(box1[3], box2[3])

    # if there is no intersection, return 0
    if x_right < x_left or y_bottom < y_top:
        return 0.0

    # calculate the area of intersection rectangle
    intersection_area = (x_right - x_left) * (y_bottom - y_top)

    # calculate the area of both boxes
    box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
    box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])

    # calculate the IOU
    iou = intersection_area / float(box1_area + box2_area - intersection_area)

    return iou


def compute_iou_lookup(pred_boxes, gt_boxes):
    """
    Compute a lookup table of IOU values between predicted and ground truth bounding boxes.
    pred_boxes: list of predicted bounding boxes
    gt_boxes: list of ground truth bounding boxes
    """
    # convert boxes to numpy arrays for faster computation
    pred_boxes = np.array(pred_boxes)
    gt_boxes = np.array(gt_boxes)

    # determine the number of predicted and ground truth boxes
    num_pred_boxes = len(pred_boxes)
    num_gt_boxes = len(gt_boxes)

    # initialize IOU lookup table with zeros
    iou_lookup = np.zeros((num_pred_boxes, num_gt_boxes))

    # calculate IOU between all pairs of boxes
    for i in range(num_pred_boxes):
        box1 = pred_boxes[i]
        for j in range(num_gt_boxes):
            box2 = gt_boxes[j]
            iou = calculate_iou(box1, box2)
            iou_lookup[i, j] = iou

    return iou_lookup

def calculate_precision_recall(pred_boxes, gt_boxes, iou_thresh, iou_lookup):
    """
    Calculate precision and recall of predicted bounding boxes with respect to the ground truth boxes.
    pred_boxes: list of predicted bounding boxes
    gt_boxes: list of ground truth bounding boxes
    iou_thresh: IOU threshold for considering a prediction as correct
    iou_lookup: pre-calculated IOU lookup table between predicted and ground truth boxes
    """
    # convert boxes to numpy arrays for faster computation
    pred_boxes = np.array(pred_boxes)
    gt_boxes = np.array(gt_boxes)

    # handle cases with no predicted or no ground truth boxes
    if len(pred_boxes) == 0:
        return 0, 0
    if len(gt_boxes) == 0:
        return 0, 0

    # count true positives, false positives, and false negatives
    tp = 0
    fp = 0
    fn = 0
    for i in range(len(pred_boxes)):
        iou_max = np.max(iou_lookup[i])
        if iou_max >= iou_thresh:
            tp += 1
        else:
            fp += 1
    for j in range(len(gt_boxes)):
        iou_max = np.max(iou_lookup[:, j])
        if iou_max < iou_thresh:
            fn += 1

    # calculate precision and recall
    if tp + fp == 0:
        precision = 0
    else:
        precision = tp / (tp + fp)
    if tp + fn == 0:
        recall = 0
    else:
        recall = tp / (tp + fn)
    return precision, recall

### Input Files

In [6]:
pred_dir = './../../results/ocr/dbnet_vgg16/txt/'
gt_dir = './../../data/ocr/docbank/txt/'

iou_threshold = 0.5

In [7]:
precisions = np.array([])
recalls = np.array([])
for file in tqdm(os.listdir(pred_dir)):
    df = pd.read_csv(pred_dir + file, sep=' ')
    gt_df = pd.read_csv(gt_dir + file, sep=' ')

    preds = df[['X1','Y1','X2','Y2']].values.tolist()
    gts =  gt_df[['X1','Y1','X2','Y2']].values.tolist()
    p,r = calculate_precision_recall(preds, gts, iou_threshold, compute_iou_lookup(preds, gts))
    precisions = np.append(precisions,p)
    recalls = np.append(recalls,r)

print(np.mean(precisions), np.mean(recalls))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:41<00:00,  2.39it/s]

0.8623650352509504 0.7553533253143856


In [20]:
import math

In [26]:
crrs = np.array([])
wrrs = np.array([])

for file in tqdm(os.listdir(pred_dir)):
    df = pd.read_csv(pred_dir + file, sep=' ')
    gt_df = pd.read_csv(gt_dir + file, sep=' ')
    
    y = gt_df['token'].to_list()
    x = df['token'].to_list()

    try:
        pred_text = ' '.join(x)
    except:
        x = [a for a in x if isinstance(a,str)]
        pred_text = ' '.join(x)
    
    try:
        gt_text = ' '.join(y)
    except:
        y = [x for x in y if isinstance(x,str)]
        gt_text = ' '.join(y)

    CRR = 100 - fastwer.score_sent(pred_text, gt_text, char_level=True)
    WRR = 100 - fastwer.score_sent(pred_text, gt_text)
    crrs = np.append(crrs, CRR)
    wrrs = np.append(wrrs, WRR)

print("avg_CRR:", np.mean(crrs))
print("avg_WRR:", np.mean(wrrs))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.15it/s]

avg_CRR: 64.854679
avg_WRR: 55.802972


In [ ]:
hypo = ['This is an example .', 'This is another example .']
ref = ['This is the example :)', 'That is the example .']

hypo = g

# Corpus-Level WER: 40.0
print(fastwer.score(hypo, ref))
# Corpus-Level CER: 25.5814
print(fastwer.score(hypo, ref, char_level=True))

# Sentence-Level WER: 40.0
print(fastwer.score_sent(hypo[0], ref[0]))
# Sentence-Level CER: 22.7273
print(fastwer.score_sent(hypo[0], ref[0], char_level=True))